In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

# Import for your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update username, password, server, port, and CRUD Python module name.
# Replace the following values with your actual credentials and server information.
username = 'aacuser'
password = 'Emerson7'
CRUD = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 30123)

# Class read method must support return of list object and accept projection JSON input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(CRUD.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the DataFrame here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new DataFrame that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Christine Emerson')

# Load and encode the image for the logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout of the dashboard
app.layout = html.Div([
    # Hidden div to store intermediate data (if needed)
    html.Div(id='hidden-div', style={'display': 'none'}),
    
    # Left-side logo
    html.Img(
        id='customer-image',
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        alt='customer image',
        style={
            'width': '5%',
            'float': 'left',
            'position': 'relative',
            'padding-top': 0,
            'padding-left': 0
        }
    ),
    
    # Right-side logo
    html.Img(
        id='customer-image',
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        alt='customer image',
        style={
            'width': '5%',
            'float': 'right',
            'position': 'relative',
            'padding-top': 0,
            'padding-right': 0
        }
    ),
    
    # Header and subtitle
    html.Center(html.B(html.H1('Austin Animal Center Outcomes'))),
    html.Center(html.B(html.H3('by Christine Emerson'))),
    html.Hr(),
    
    # Radio items for filtering animal type
    html.Div([
        html.B("Filter Animal Type:"),
        html.Br(),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Dog ', 'value': 'dog'},
                {'label': 'Cat ', 'value': 'cat'},
                {'label': 'Bird ', 'value': 'bird'},
                {'label': 'Other ', 'value': 'other'},
                {'label': 'Reset ', 'value': 'reset'},
            ],
            value='reset'
        ),
    ]),
    html.Hr(),
    
    # DataTable for displaying the animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True}
            for i in df.columns
        ],
        data=df.to_dict('records'),

        # Added features to make it more user-friendly for the client

        # Editing the cells
        editable=True,
        # Filtering by column
        filter_action='native',
        # Sorting by column
        sort_action='native',
        # Enable deletion of rows
        row_deletable=True,
        sort_mode='multi',
        # Enable single-row selection for the data table for the mapping callback to function properly.
        row_selectable='single',
        # Selecting columns
        selected_columns=[],
        # Selecting rows
        selected_rows=[0],
        # Paging front-end
        page_action='native',
        # Current page 0
        page_current=0,
        # Animals per page
        page_size=10,
        # Hiding columns
        hidden_columns=[],
    ),
    html.Br(),
    html.Hr(),
    
    # Map for displaying the location of the selected animal
    html.Div(id='map-id', className='col s12 m6')
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter callback update
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Updates the data table

    # Dog filter toggled
    if filter_type == 'dog':
        df = pd.DataFrame.from_records(CRUD.read({'animal_type': 'Dog'}))
    # Cat filter toggled
    elif filter_type == 'cat':
        df = pd.DataFrame.from_records(CRUD.read({'animal_type': 'Cat'}))
    # Bird filter toggled
    elif filter_type == 'bird':
        df = pd.DataFrame.from_records(CRUD.read({'animal_type': 'Bird'}))
    # Other filter toggled
    elif filter_type == 'other':
        df = pd.DataFrame.from_records(CRUD.read({'animal_type': 'Other'}))
    # Defaults to reset being selected
    else:
        # Resets the query to no filter and gets all the data
        df = pd.DataFrame.from_records(CRUD.read({}))

    df.drop(columns=['_id'], inplace=True)  # Drop the _id column from the DataFrame

    columns = [
        {'name': i, 'id': i, 'deletable': False, 'selectable': True}
        for i in df.columns
    ]
    data = df.to_dict('records')

    # Returns the output data for the data table
    return data, columns


# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the data table in the form of
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection, so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the data table
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)

    # Check if derived_virtual_data is None or empty
    if viewData is None or not viewData:
        return dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id='base-layer-id')
            ]
        )

    if index is None or not index:
        row = 0
    else:
        row = index[0]

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          dl.Popup([
                              html.H1('Animal Name'),
                              html.P(dff.iloc[row, 9])
                          ])
                      ])
        ]
    )

app.run_server(debug=True)


My Mongo server is nv-desktop-services.apporto.com
My Mongo port is 30123
10014
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', 'animal id', 'namee'],
      dtype='object')
Dash app running on http://127.0.0.1:19981/
